In [1]:
import os
from os.path import isdir, join
from pathlib import Path
import pandas as pd

# Math
import numpy as np
from scipy.fftpack import fft
from scipy import signal
from scipy.io import wavfile
import librosa

from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
import seaborn as sns
import IPython.display as ipd
import librosa.display

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import pandas as pd

%matplotlib inline

In [2]:

dirs = [f for f in os.listdir('/Users/samanyukamra/Dropbox/Mac/Documents/downloadss/archive (1)/data/training_data') if isdir(join('/Users/samanyukamra/Dropbox/Mac/Documents/downloadss/archive (1)/data/training_data', f))]
dirs.sort()
print('Number of labels: ' + str(len(dirs)))

Number of labels: 36


In [3]:

dirs_1 = [f for f in os.listdir('/Users/samanyukamra/Dropbox/Mac/Documents/downloadss/archive (1)/data/testing_data') if isdir(join('/Users/samanyukamra/Dropbox/Mac/Documents/downloadss/archive (1)/data/testing_data', f))]
dirs_1.sort()
print('Number of labels: ' + str(len(dirs_1)))

Number of labels: 36


In [4]:
base_dir = '/Users/samanyukamra/Dropbox/Mac/Documents/downloadss/archive (1)/data/training_data'

# Lists to store file paths and labels
file_paths = []
labels = []

# Loop through each subfolder
for folder in os.listdir(base_dir):
    if folder != '_background_noise_' and os.path.isdir(os.path.join(base_dir, folder)):
        # Loop through each file in the subfolder
        for file in os.listdir(os.path.join(base_dir, folder)):
            # Add the file path to the list
            file_paths.append(os.path.join(base_dir, folder, file))
            # Add the label (folder name) to the list
            labels.append(folder)

In [5]:
from sklearn.model_selection import train_test_split

# Split data into training and validation sets
# Test size can be adjusted. 0.2 means 20% of the data will be used for validation.
X_train, X_val, y_train, y_val = train_test_split(file_paths, labels, test_size=0.1, random_state=24)

In [6]:
print(len(X_train))
print(len(y_train))
print(len(X_val))
print(len(y_val))

18565
18565
2063
2063


In [7]:
print(X_train[18])
print(X_val[18])
print(y_train[18])
print(y_val[18])

/Users/samanyukamra/Dropbox/Mac/Documents/downloadss/archive (1)/data/training_data/Y/1618.png
/Users/samanyukamra/Dropbox/Mac/Documents/downloadss/archive (1)/data/training_data/P/13093.png
Y
P


In [8]:
test_dir = '/Users/samanyukamra/Dropbox/Mac/Documents/downloadss/archive (1)/data/testing_data'

In [9]:
X_test = []
y_test = []

for subdir in os.listdir(test_dir):
    # Create the full path to the subdirectory
    subdir_path = os.path.join(test_dir, subdir)
    
    # Check if this path is indeed a directory
    if os.path.isdir(subdir_path):
        # If it is a directory, iterate through all the files in that directory
        for file in os.listdir(subdir_path):
            # Create the full path to the file and append it to X_test
            file_path = os.path.join(subdir_path, file)
            X_test.append(file_path)
            # Append the directory name (label) to y_test
            y_test.append(subdir)  # subdir is the directory name, which is used as the label


In [10]:
print((y_test[100]))
print(X_test[9])

0
/Users/samanyukamra/Dropbox/Mac/Documents/downloadss/archive (1)/data/testing_data/R/28805.png


In [12]:
pip install opencv-python


  Obtaining dependency information for opencv-python from https://files.pythonhosted.org/packages/a1/f6/57de91ea40c670527cd47a6548bf2cbedc68cec57c041793b256356abad7/opencv_python-4.8.1.78-cp37-abi3-macosx_11_0_arm64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.1/33.1 MB 1.4 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [11]:
import cv2
import os
import numpy as np
from skimage.feature import hog
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Constants
IMAGE_SIZE = (28, 28)  # Standard image size

def preprocess_image(image_path):
    # Read and resize image
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, IMAGE_SIZE)

    # Normalize pixel values
    img = img / 255.0

    return img

def extract_features(image):
    # Extract HOG features
    features, _ = hog(image, orientations=8, pixels_per_cell=(4, 4),
                      cells_per_block=(1, 1), visualize=True)
    return features




In [12]:
def process_images(file_paths):
    features_list = []

    for file_path in file_paths:
        image = preprocess_image(file_path)
        features = extract_features(image)
        features_list.append(features)

    return np.array(features_list)


In [13]:

train_features = process_images(X_train)
val_features = process_images(X_val)
test_features = process_images(X_test)  # No y_test needed

In [14]:
train_labels = np.array(y_train)
val_labels = np.array(y_val)
test_labels = np.array(y_test)

In [15]:
print((test_labels[50]))

U


In [16]:

# Scale features
scaler = StandardScaler()
train_features_scaled = scaler.fit_transform(train_features)
val_features_scaled = scaler.transform(val_features)
test_features_scaled = scaler.transform(test_features)


In [17]:
print(len(test_features_scaled))

1008


In [18]:
from sklearn.cluster import KMeans
import numpy as np

# Assuming you know the number of classes (for example, 10 digits for a digit recognition task)
n_clusters = 36

# Initialize KMeans with the number of clusters
kmeans = KMeans(n_clusters=n_clusters, random_state=42)

# Fit the model on the scaled training data
kmeans.fit(train_features_scaled)



/Users/samanyukamra/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



KMeans(n_clusters=36, random_state=42)

In [19]:
train_clusters = kmeans.predict(train_features_scaled)

In [20]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

# Assuming y_train and y_val contain string labels
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)





In [21]:
label_counts_per_cluster = np.zeros((kmeans.n_clusters, len(label_encoder.classes_)), dtype=int)

for cluster in range(kmeans.n_clusters):
    indices = np.where(train_clusters == cluster)[0]
    labels, counts = np.unique(y_train_encoded[indices], return_counts=True)
    label_counts_per_cluster[cluster][labels] = counts

cluster_to_label_mapping = np.argmax(label_counts_per_cluster, axis=1)


In [22]:
from sklearn.metrics import confusion_matrix
import numpy as np
from sklearn.metrics import accuracy_score

val_predicted_labels = cluster_to_label_mapping[kmeans.predict(val_features_scaled)]
val_accuracy = accuracy_score(y_val_encoded, val_predicted_labels)
print(f'Validation Set Accuracy: {val_accuracy * 100:.2f}%')


Validation Set Accuracy: 76.10%


In [23]:
label_encoder =  LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

In [24]:
label_counts_per_cluster = np.zeros((kmeans.n_clusters, len(label_encoder.classes_)), dtype=int)

for cluster in range(kmeans.n_clusters):
    indices = np.where(train_clusters == cluster)[0]
    labels, counts = np.unique(y_train_encoded[indices], return_counts=True)
    label_counts_per_cluster[cluster][labels] = counts

cluster_to_label_mapping = np.argmax(label_counts_per_cluster, axis=1)


In [26]:
from sklearn.metrics import confusion_matrix
import numpy as np
from sklearn.metrics import accuracy_score

from sklearn.metrics import classification_report, accuracy_score

test_predicted_labels = cluster_to_label_mapping[kmeans.predict(test_features_scaled)]
test_accuracy = accuracy_score(y_test_encoded, test_predicted_labels)
print(f'Test Set Accuracy: {test_accuracy * 100:.2f}%')

print("Classification Report:\n", classification_report(y_test_encoded, test_predicted_labels))

Test Set Accuracy: 85.32%
Classification Report:
               precision    recall  f1-score   support

           0       0.49      1.00      0.66        28
           1       1.00      1.00      1.00        28
           2       1.00      1.00      1.00        28
           3       1.00      1.00      1.00        28
           4       1.00      1.00      1.00        28
           5       1.00      1.00      1.00        28
           6       1.00      1.00      1.00        28
           7       1.00      1.00      1.00        28
           8       0.50      1.00      0.67        28
           9       1.00      1.00      1.00        28
          10       1.00      1.00      1.00        28
          11       0.00      0.00      0.00        28
          12       0.50      1.00      0.67        28
          13       1.00      0.96      0.98        28
          14       0.50      1.00      0.67        28
          15       0.00      0.00      0.00        28
          16       0.00      0.

/Users/samanyukamra/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/Users/samanyukamra/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/Users/samanyukamra/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [27]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

# Create an instance of the SVC (Support Vector Classifier)
svm_model = SVC(gamma='auto')

# Train the model using the scaled training data and labels
svm_model.fit(train_features_scaled, train_labels)

# Validate the model using the scaled validation data and labels
val_predictions = svm_model.predict(val_features_scaled)

# Evaluate the performance
print("Validation Accuracy:", accuracy_score(val_labels, val_predictions))
print("Classification Report:\n", classification_report(val_labels, val_predictions))



Validation Accuracy: 0.971885603490063
Classification Report:
               precision    recall  f1-score   support

           0       0.63      0.81      0.71        42
           1       0.96      0.93      0.95        59
           2       0.98      1.00      0.99        52
           3       1.00      1.00      1.00        52
           4       0.98      1.00      0.99        54
           5       1.00      0.98      0.99        55
           6       1.00      1.00      1.00        60
           7       0.98      1.00      0.99        42
           8       1.00      0.99      0.99        72
           9       0.98      1.00      0.99        58
           A       0.98      0.97      0.98        62
           B       1.00      0.98      0.99        46
           C       0.98      1.00      0.99        45
           D       0.95      1.00      0.97        56
           E       1.00      0.96      0.98        52
           F       0.98      0.97      0.98        65
           G      

In [29]:
test_predictions = svm_model.predict(test_features_scaled)
print(test_predictions)
print("Validation Accuracy:", accuracy_score(test_labels, test_predictions))
print("Classification Report:\n", classification_report(test_labels, test_predictions))


['R' 'R' 'R' ... 'W' 'W' 'W']
Validation Accuracy: 0.9811507936507936
Classification Report:
               precision    recall  f1-score   support

           0       0.65      0.71      0.68        28
           1       1.00      1.00      1.00        28
           2       1.00      1.00      1.00        28
           3       1.00      1.00      1.00        28
           4       1.00      1.00      1.00        28
           5       1.00      1.00      1.00        28
           6       1.00      1.00      1.00        28
           7       1.00      1.00      1.00        28
           8       1.00      1.00      1.00        28
           9       1.00      1.00      1.00        28
           A       1.00      1.00      1.00        28
           B       1.00      1.00      1.00        28
           C       1.00      1.00      1.00        28
           D       1.00      1.00      1.00        28
           E       1.00      1.00      1.00        28
           F       1.00      1.00      1.